In [31]:
import geopandas
import pandas as pd
import numpy as np
import random
import math
from math import sqrt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from shapely.geometry import Point

In [32]:
persons_df = pd.read_feather('data/trips_others.feather')

In [33]:
persons_df.chain_length.sum()

3311053

In [34]:
import folium
import geopandas
from shapely.geometry import Point

In [35]:
# persons_df['home'] = persons_df.home.apply(lambda x: Point(x))
# persons_df['coords'] = persons_df.coords.apply(lambda x: [Point(y) for y in x])

In [38]:
persons_df

,index,id,age,Ruta,Rutstorl,home,trip_chain,chain_length,home_ruta_storlek,coords
0,0,0,Ald7_15,3495006225750,250,"[12.577003233474832, 56.15371189050657]",[],0,3495006225750250,"[[12.577003233474832, 56.15371189050657]]"
1,1,1,Ald7_15,3495006225750,250,"[12.577639938001655, 56.15307599069771]","[4, 4, 1]",3,3495006225750250,"[[12.577639938001655, 56.15307599069771], [12...."
2,2,2,Ald7_15,3495006225750,250,"[12.578197194684574, 56.15334745858721]","[4, 4, 1]",3,3495006225750250,"[[12.578197194684574, 56.15334745858721], [12...."
3,3,3,Ald7_15,3495006225750,250,"[12.578197194684574, 56.15334745858721]","[4, 1]",2,3495006225750250,"[[12.578197194684574, 56.15334745858721], [12...."
4,4,4,Ald7_15,3495006225750,250,"[12.579965306016872, 56.15518771277077]","[4, 1, 16, 1]",4,3495006225750250,"[[12.579965306016872, 56.15518771277077], [12...."
...,...,...,...,...,...,...,...,...,...,...
1311072,1311072,1576774,Ald65_W,4320006189000,1000,"[13.917585218973997, 55.85007009496898]","[7, 7, 12]",3,43200061890001000,"[[13.917585218973997, 55.85007009496898], [13...."
1311073,1311073,1576775,Ald65_W,4320006189000,1000,"[13.922698331888617, 55.84516290224753]",[],0,43200061890001000,"[[13.922698331888617, 55.84516290224753]]"
1311074,1311074,1576779,Ald7_15,4310006244000,1000,"[13.893047083591613, 56.335926489142615]","[4, 1, 11, 1]",4,43100062440001000,"[[13.893047083591613, 56.335926489142615], [13..."
1311075,1311075,1576780,Ald7_15,4310006244000,1000,"[13.88484608563395, 56.34245536086377]",[],0,43100062440001000,"[[13.88484608563395, 56.34245536086377]]"


In [39]:
kommun = geopandas.read_file('data/kommuns.geojson')
kommun = kommun.to_crs('EPSG:4326')

In [40]:
# persons_df = persons_df.sample(100000)

In [41]:
import itertools
flat_chain = list(itertools.chain(*persons_df.coords.values.tolist()))
# flat_chain = list(set(flat_chain))

In [42]:
len(flat_chain)

353262

In [43]:
coords_kommun = []
for coord, i in zip(flat_chain, range(len(flat_chain))):
    coords_kommun.append(kommun[kommun.contains(Point(coord))].index[0])

In [44]:
pd.DataFrame(coords_kommun, columns=['kommun']).to_feather('data/coords_kommun.feather')

In [45]:
coords_kommun = pd.read_feather('data/coords_kommun.feather').kommun.to_list()

In [46]:
len(coords_kommun)

353262

In [47]:
ks = (x for x in coords_kommun)
kf = (x for x in coords_kommun[1:])
dd = {}
for person in persons_df.itertuples():
#     print(person.id, person.coords)
    for sd, fd, skomm, fkomm in zip(person.coords, person.coords[1:], ks, kf):
#         print(sd, fd)
        if (skomm,fkomm) in dd:
            dd[(skomm,fkomm)] += 1
        elif (fkomm,skomm) in dd:
            dd[(fkomm,skomm)] += 1
        else:
            dd[(skomm,fkomm)] = 1
    try:
        next(ks)
        next(kf)
    except StopIteration:
        print('finnished')
        pass

finnished


In [54]:
dd

{(50, 50): 5241,
 (1, 1): 2621,
 (49, 37): 649,
 (37, 18): 508,
 (18, 49): 467,
 (46, 57): 399,
 (57, 27): 1279,
 (27, 3): 109,
 (3, 46): 82,
 (30, 30): 38899,
 (29, 29): 18775,
 (18, 18): 12197,
 (39, 39): 4392,
 (15, 15): 3119,
 (57, 30): 4374,
 (0, 0): 1489,
 (32, 20): 914,
 (32, 32): 3425,
 (55, 55): 1221,
 (22, 57): 249,
 (46, 29): 1941,
 (37, 37): 1048,
 (40, 40): 1417,
 (38, 40): 289,
 (26, 9): 27,
 (42, 23): 453,
 (23, 23): 1913,
 (55, 22): 89,
 (22, 39): 1128,
 (39, 55): 146,
 (31, 31): 1907,
 (13, 13): 959,
 (13, 6): 44,
 (13, 18): 13,
 (13, 50): 203,
 (9, 9): 7421,
 (50, 15): 390,
 (42, 42): 3541,
 (42, 1): 437,
 (22, 22): 5461,
 (30, 20): 2148,
 (32, 27): 357,
 (27, 27): 3946,
 (30, 22): 1517,
 (30, 39): 2766,
 (12, 12): 3178,
 (30, 9): 150,
 (49, 49): 2174,
 (55, 30): 2865,
 (29, 50): 1295,
 (57, 57): 14539,
 (6, 6): 1532,
 (0, 15): 29,
 (0, 12): 494,
 (12, 48): 586,
 (46, 46): 4919,
 (38, 18): 799,
 (42, 57): 165,
 (29, 15): 1478,
 (57, 20): 1617,
 (27, 31): 544,
 (31, 39

In [67]:
summ = sum([d for d in dd.values()])

In [71]:
c=500

In [72]:
COORD = [55.63135, 13.70622]
m = folium.Map(location=COORD, zoom_start=8)

for k,tr in dd.items():
    if -1 in k:
        continue
    s = kommun.loc[k[0]].geometry.centroid
    e = kommun.loc[k[1]].geometry.centroid
    if s == e:
        folium.Circle((s.coords[0][1], s.coords[0][0]), weight=tr/summ*c, tooltip=str(tr)).add_to(m)
    else:
        folium.PolyLine([(x.coords[0][1], x.coords[0][0]) for x in [s,e]], weight=tr/summ*c, tooltip=str(tr)).add_to(m)
m.save('data/demand.html')